# Bollenger Band Algo

In [1]:
import numpy as np
import pandas as pd
from pylab import plt
from sklearn import linear_model
plt.style.use('seaborn')
#matplotlib inline

data_url = 'http://hilpisch.com/tr_eikon_eod_data.csv'
raw = pd.read_csv(data_url, index_col=0,parse_dates=True)
raw.dropna(inplace=True)

symbol = 'AAPL.O'

data = pd.DataFrame(raw[symbol])

data['Returns'] = np.log(data[symbol] / data[symbol].shift(1))
data.dropna(inplace=True)

data['Direction'] = np.sign(data['Returns'])



In [ ]:
data[symbol].plot(figsize=(15,10), title=symbol)

In [ ]:
# so now we've done all the default crap, lets see about running a BB on this mofo

In [3]:
#next w define a function to do the bollenger band calc
def genBollengerChannels(duration):
    
    # first get the average over the window
    
    data['Strategy_m'] = data[symbol].rolling(window=duration).mean()
    data.dropna(inplace=True)
    
    data['Strategy_h'] = data['Strategy_m'] + (data[symbol].rolling(window=duration).std() * 2)
    data.dropna(inplace=True)
    data['Strategy_l'] = data['Strategy_m'] - (data[symbol].rolling(window=duration).std() * 2)
    data.dropna(inplace=True)
    
    data['MeanReturns_m'] = data['Returns'].rolling(window=duration).mean()
    data.dropna(inplace=True)
    
    data['MeanReturns_h'] = data['MeanReturns_m'] + (data[symbol].rolling(window=duration).std() * 2 )
    data.dropna(inplace=True)
    
    data['MeanReturns_l'] = data['MeanReturns_m'] - (data[symbol].rolling(window=duration).std() * 2 )
    data.dropna(inplace=True)
    

In [4]:
def genSignals():
    
    data['Signal'] = []
    
# I need to make this a single line and have the shift(1) in there
    data['Signal'] = np.where((data[symbol] > data['Strategy_l']) and data['Signal'].shift(1), 1, -1)
#   data['Signal'] = np.where(data[symbol] < data['Strategy_h'], -1, 1)
    
    data['Strategy'] = data['Signal'].shift(1) * data['Returns']
    data.dropna(inplace=True)

    data['Direction'] = np.sign(data['Returns'])

In [5]:
genBollengerChannels(5)

In [6]:
genSignals()

ValueError: Length of values does not match length of index

In [ ]:
hit_ratio = data['Signal'].shift(1) == data['Direction']

In [9]:
data[[symbol,'Returns']]

,AAPL.O,Returns
Date,,
2010-02-09,28.027115,0.010607
2010-02-10,27.873686,-0.005489
2010-02-11,28.381400,0.018051
2010-02-12,28.625686,0.008570
2010-02-16,29.057114,0.014959
2010-02-17,28.935685,-0.004188
2010-02-18,28.989685,0.001864
2010-02-19,28.809971,-0.006219
2010-02-22,28.630829,-0.006237


In [ ]:
hit_ratio.value_counts()

In [ ]:
data[[symbol,'Strategy_m','Strategy_h','Strategy_l']][1750:].plot(figsize=(15,10), title=symbol)

In [ ]:
data[['Returns','Strategy']].cumsum().apply(np.exp).plot(figsize=(15,10))

In [ ]:

hit_ratio